In [1]:
import requests

from elasticsearch import Elasticsearch
import tiktoken
from tqdm.auto import tqdm

In [2]:
elastic_search_client = Elasticsearch('http://localhost:9200')

## Q1

In [3]:
elastic_search_client.info()

ObjectApiResponse({'name': 'ad15fa4654dc', 'cluster_name': 'docker-cluster', 'cluster_uuid': '05BJz8gkSReW0GAKSqVkZg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Q2

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
        },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'index-for-homework'

elastic_search_client.indices.create(index=index_name, body=index_settings)

for one_document in tqdm(documents):
    elastic_search_client.index(index=index_name, document=one_document)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3

In [5]:
question = 'How do I execute a command in a running docker container?'

index_name = 'course-questions'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question, 
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
                },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
                },
            }
        },
    }
elastic_search_results = elastic_search_client.search(index=index_name, body=search_query)
search_results = [one_hit["_source"] for one_hit in elastic_search_results["hits"]["hits"]]

In [6]:
elastic_search_results["hits"]["hits"][0]['_score']

75.54128

## Q4

In [7]:
question = 'How do I execute a command in a running docker container?'

index_name = 'course-questions'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question, 
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
                },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
                },
            }
        },
    }
elastic_search_results = elastic_search_client.search(index=index_name, body=search_query)
search_results = [one_hit["_source"] for one_hit in elastic_search_results["hits"]["hits"]]

In [8]:
search_results[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

## Q5

In [9]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [10]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [11]:
context = '\n\n'.join(
    [context_template.format(question=one_search_result['question'], text=one_search_result['text']) for one_search_result in search_results]
    )

In [12]:
prompt = prompt_template.format(question=question, context=context)

In [13]:
len(prompt)

2714

## Q6

In [14]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [15]:
len(encoding.encode(prompt))

621